# Instalacja solvera Clingo

Według oficjalnych [instrukcji](https://github.com/potassco/clingo/releases)

Oczywiście, instalacja dla systemów Windows oraz MacOS będzie się różnić.

In [ ]:
!add-apt-repository -y ppa:potassco/stable > /dev/null
!apt install clingo > /dev/null

# Weryfikacja poprawności działania
!clingo -v

# Definiujemy nowe "cell magic" dla uproszczenia uruchamiania clingo
import re
from IPython import get_ipython
from IPython.display import HTML
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.core import magic_arguments

def syntax_highlight(text):
    rules = {
        'color:orange': r'\b(Answer:\s\d+|SATISFIABLE)\b',
        'color:green': r'\b(SATISFIABLE)\b',
        'color:red': r'\b(UNSATISFIABLE)\b',
    }
    for style, pattern in rules.items():
        text = re.sub(pattern, r'<span style="{}">\1</span>'.format(style), text)
    return f'<pre>{text}</pre>'

@magics_class
class ClingoMagic(Magics):
    @cell_magic
    def clingo(self, line, cell):
        with open('temp.lp', 'w') as fp:
            fp.write(cell)
        out = '\n'.join(self.shell.getoutput(f'clingo {line} temp.lp'))
        highlighted = syntax_highlight(out)
        display(HTML(highlighted))

ipy = get_ipython()
ipy.register_magics(ClingoMagic)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


clingo version 5.8.0 (6d1efb6)
Address model: 64-bit

libclingo version 5.8.0
Configuration: with Python 3.10.12, with Lua 5.3.6

libclasp version 3.4.0 (libpotassco version 1.2.0)
Configuration: WITH_THREADS=1
Copyright (C) Benjamin Kaufmann

License: The MIT License <https://opensource.org/licenses/MIT>


# Wprowadzenie

Answer set programming lub *dysjunkcyjne
programowanie logiczne ze stabilnymi modelami* jest formą programowania logicznego dla rozwiązywania trudnych problemów kombinatorycznych.

Programy zapisujemy w języku `AnsProlog` i składają się one z zestawu
**reguł** oraz opcjonalnych **dykrektyw**.

Do uruchamiania programów w `AnsProlog` będziemy używali solwera `clingo`

In [3]:
%%clingo


imie(ania).
imie(bartek).
imie("Adam").

numer(1). numer(2). numer(7).
numer(3 ; 4 ; 5 ; 7).
numer(1..10).

wzrost(bartek, 180).
wzrost(ania, 170).
wzrost("Adam", 195).

wysoka(Osoba) :- wzrost(Osoba, H), H >= 190.
niska(Osoba) :- wzrost(Osoba, H), not wysoka(Osoba).

#show wysoka/1.
#show niska/1.

UsageError: Cell magic `%%clingo` not found.


In [ ]:
%%clingo -n 0

el(1..5).


{ wybrana(X) : el(X)  } = 3.

## Predykaty

Reguły w programach składać się będą z:
- predykatów
- stałych
- wyrażeń arytm.

**Predykat** opisuje właściwość lub relację, którą będziemy stosować w programie.
Nazwy predykatów zapisujemy od małej litery i są one dowolne.

Uwaga, polskie znaki nie są obsługiwane.

**Stałe** to liczby **całkowite** oraz ciągi liter rozpoczynające się od małej litery.

In [ ]:
%%clingo -n 0

imie(jola).
imie(bartek).
imie("Adam").

answer(42).

kraj(polska).
kraj(niemcy).
kraj(litwa ; czechy).     %# Skrót: zamiast kraj(litwa). kraj(czechy).

sasiedzi(polska , niemcy).  %# Predykat z 2 argumentami -- przecinek "," rozdziela argumenty.
sasiedzi(polska , litwa).
sasiedzi(niemcy, czechy).
sasiedzi(polska, czechy).

kolor(niebieski ; czerwony ; zielony).

{ pokoloruj(X, K) : kolor(K) } = 1 :- kraj(X).

:- pokoloruj(Kraj1, K), pokoloruj(Kraj2, K), sasiedzi(Kraj1, Kraj2).

#show pokoloruj/2.


## Reguły i zmienne

Reguły mają składnię:

    głowa reguły :- ciało reguły .

Znaczenie reguły jest bliskie znaczenia **implikacji** w logice zdań, czyli
ciało reprezentuje *przesłanki* a głowa to *konkluzja*.

Reguła jest **spełniona** poza przypadkiem gdy z prawdziwej przesłanki wynika fałszywa konkluzja.

### Zmienne

Zapisywane są z wielkiej litery i w ich miejsce podstawiane
są (przez solwer) stałe wynikające z predykatów.

Zmienne przydatne są w *zapisie* reguł.

In [ ]:
%%clingo

%# Dyrektywa do zapisu stałej liczbowej:

#const n=10.

liczba(1..n).  %# Zamiast liczba(1). liczba(2). aż do liczba(n).

%# Przecinek w ciele reguły oznacza spójnik AND.
%# Regułę możemy odczytać jako:
%#    X jest odpowiedzią, jeżeli (X jest liczbą) AND (2*x = 10)

odpowiedz(X) :- liczba(X) , 2*X == 20 .

#show odpowiedz/1.

### Dyrektywa `show`

Dzięki dyrektywie `show` możemy ograniczyć wyświetlanie tylko do interesujących nas predykatów.

In [ ]:
%%clingo

#const n=10.

liczba(1..n).  %# Zamiast liczba(1). liczba(2). aż do liczba(n).

odpowiedz(X) :- liczba(X) , 2*X == 20 .

parzysta(X) :- liczba(X) , (X / 2) * 2 == X .

nastepna(A, B) :- liczba(A), liczba(B), A + 1 == B.

#show odpowiedz/1.
#show parzysta/1.
#show nastepna/2.

## Anonimowa zmienna

W przypadku, gdy wartość zmiennej nie ma znaczenia, tj. nie występuje nigdzie w innych częściach reguły, to możemy zastosować zmienną anonimową `_` (symbol podkreślenia)

In [25]:
%%clingo

malzonkowie(ewa, adam).
malzonkowie(zofia, stefan).

zona(Z) :- malzonkowie(Z, _).
maz(M) :- malzonkowie(_ , M).

#show zona/1.
#show maz/1.

## Lewa strona reguły

Tylko jedna z klauzul występujących po *lewej* stronie reguły jest prawdziwa,
tj. przecinek możemy traktować jako spójnik **"albo"**.

Przecinek między klauzulamy po *prawej* stronie reguły interpretujemy jako spójnik **"i"** (and).

In [ ]:
%%clingo -n 0

polska, niemcy :- .

%# lub krócej:

polska, niemcy.

%# ok nie zostanie wywnioskowane:
ok :- 1 > 0, 1 > 2.

# Zad. 1.

Podana jest lista krajów, proszę dodać regułę, która znajdzie *duże* kraje, tj.
o liczbie ludności większej niż 60. Odpowiedź powinna być w formie predykatu
`duzy_kraj(nazwa kraju)`

Proszę skorzystać z podanej listy predykatów dotyczących populacji wybranych
krajów:

    populacja(austria, 9).
    populacja(belgia, 12).
    populacja(czechy, 11).
    populacja(francja, 68).
    populacja(grecja, 10).
    populacja(hiszpania, 48).
    populacja(holandia, 18).
    populacja(polska, 39).
    populacja(portugalia, 10).
    populacja(rosja, 145).
    populacja(turcja, 85).
    populacja(ukraina, 37).
    populacja(wielka_brytania, 67).

In [8]:
%%clingo
populacja(austria, 9).
    populacja(belgia, 12).
    populacja(czechy, 11).
    populacja(francja, 68).
    populacja(grecja, 10).
    populacja(hiszpania, 48).
    populacja(holandia, 18).
    populacja(polska, 39).
    populacja(portugalia, 10).
    populacja(rosja, 145).
    populacja(turcja, 85).
    populacja(ukraina, 37).
    populacja(wielka_brytania, 67).

    duzy_kraj(Kraj) :- populacja(Kraj, Populacja), Populacja > 60.

    #show duzy_kraj/1.

# Zad 2.

Wykonaj program modelujący rzut *parą kości sześciościennych* do gry. Zdefiniuj
predykat dla wyniku rzutu kością oraz regułę określającą możliwą do uzyskania sumę na obu kościach.

# Zad 2b

Zmodyfikuj poprzedni program, tak aby wyznaczył jakie pary wyników na kościach dają sumę równą lub większą 10.  

# Zad. 3

Proszę znaleźć liczbę dwucyfrową (w systemie dziesiętnym), która równa
jest **trzykrotności sumy swoich cyfr**.

*Wskazówka*. Liczba dwucyfrowa o cyfrach A i B, ma wartość A*10 + B.

In [22]:
%%clingo

cyfra_dziesiatek(1..9).
cyfra_jednosci(0..9).

liczba(L) :- cyfra_dziesiatek(A), cyfra_jednosci(B),
                L = A*10 + B,
                L == 3 * (A + B).

#show liczba/1.

# Zad. 4

W poniższym programie podano listę zależności pomiędzy wybranymi modułami (bibliotekami) JavaScript.

Proszę uzupełnić program, tak aby znalazł wszystkie pakiety, od których
zależy pakiet `wrap_ansi`.

**Wskazówka**. Przydatne będzie wykorzystanie faktu, że jeżeli pakiet `A` zależy od pakietu `B`, a pakiet `B` od pakietu `C`, to pakiet `A` również
zależy od pakietu `C`.

In [24]:
%%clingo

depends(jasmine, glob).
depends(glob, foreground_child).
depends(foreground_child, cross_spawn).
depends(cross_spawn, path_key).
depends(cross_spawn, shebang_command).
depends(shebang_command, shebang_regex).
depends(cross_spawn, which).
depends(which, isexe).
depends(foreground_child, signal_exit).
depends(glob, jackspeak).
depends(jackspeak, isaacs_cliui).
depends(isaacs_cliui, string_width).
depends(string_width, eastasianwidth).
depends(string_width, emoji_regex).
depends(string_width, strip_ansi).
depends(isaacs_cliui, string_width_cjs).
depends(isaacs_cliui, strip_ansi).
depends(strip_ansi, ansi_regex).
depends(isaacs_cliui, strip_ansi_cjs).
depends(isaacs_cliui, wrap_ansi).
depends(wrap_ansi, ansi_styles).
depends(wrap_ansi, string_width).
depends(wrap_ansi, strip_ansi).
depends(isaacs_cliui, wrap_ansi_cjs).
depends(jackspeak, pkgjs_parseargs).
depends(glob, minimatch).
depends(minimatch, brace_expansion).
depends(brace_expansion, balanced_match).
depends(glob, path_scurry).
depends(path_scurry, lru_cache).
depends(path_scurry, minipass).
depends(jasmine, jasmine_core).
depends(balanced_match, glob).

answer(X) :- depends(wrap_ansi,X).


%# TODO: Proszę uzupełnić ...
#show answer/1.

# Zad. 4b

Proszę zmodyfikować program dla poprzedniego zadania, tak aby wykryć listę pakietów, których zależności tworzą **cykl**, tj. pakiet zależny jest (przechodnio) od samego siebie.

# Zad. 5

Proszę znaleźć rozwiązanie następującego [problemu](https://mathworld.wolfram.com/MarriedCouplesProblem.html). Przy **okrągłym stole** należy usadzić 3 pary małżeństw (dla uproszczenia zakładamy małżeństwa heterogeniczne), tak aby nie
siedziały obok siebie:
- osoby tej samej płci,
- małżonkowie.

Lista małżeństw:

- Lisa i Michael
- Emily i David
- Sarah i James

Dla uproszczenia można założyć, że na pierwszym miejscu siedzi `Lisa`.

In [28]:
%%clingo -n 0

%# Lista par :
married(lisa, michael).
married(emily, david).
married(sarah, james).

female(Z) :- married(Z, _).
male(M) :- married(_ , M).

seat

